# Reading IDC DICOM Whole Slide Images with TIAToolbox

<a href="https://colab.research.google.com/github/fedorov/idc-tiatoolbox/blob/main/notebooks/01_reading_idc_slides_with_tiatoolbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This notebook demonstrates how to:

1. Use `idc-index` to discover and select slide microscopy data from [NCI Imaging Data Commons (IDC)](https://portal.imaging.datacommons.cancer.gov/)
2. Download DICOM WSI files from IDC
3. Open DICOM WSIs using [TIAToolbox](https://tia-toolbox.readthedocs.io/)'s `WSIReader`
4. Extract metadata, generate thumbnails, and read regions at different resolutions
5. Visualize the same slide in the browser using IDC's SLIM viewer

**No GPU required** for this notebook.

## Installation

Run the cell below to install dependencies. **On Colab, the runtime will automatically restart** after installation to pick up the updated numpy version. After the restart, continue from the imports cell below.

In [ ]:
%pip install tiatoolbox idc-index openslide-bin "numcodecs<0.16"

# Restart runtime to pick up updated numpy (required on Colab)
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from PIL import Image

from idc_index import IDCClient
from tiatoolbox.wsicore.wsireader import WSIReader

## 1. Discover Slide Microscopy Data in IDC

IDC provides a specialized `sm_index` table with metadata about slide microscopy (pathology) images, including objective lens power, pixel spacing, and image dimensions. Let's use it to find a manageable slide for this tutorial.

In [ ]:
idc_client = IDCClient()
print(f"IDC data version: {idc_client.get_idc_version()}")

# Load the slide microscopy index
idc_client.fetch_index("sm_index")

### How many slide microscopy collections are available?

Let's see which IDC collections contain slide microscopy data and how many slides each has.

In [ ]:
sm_collections = idc_client.sql_query("""
    SELECT
        i.collection_id,
        COUNT(DISTINCT i.PatientID) as patients,
        COUNT(DISTINCT s.SeriesInstanceUID) as slides,
        MIN(s.ObjectiveLensPower) as min_power,
        MAX(s.ObjectiveLensPower) as max_power,
        ROUND(SUM(i.series_size_MB), 1) as total_size_mb
    FROM sm_index s
    JOIN index i ON s.SeriesInstanceUID = i.SeriesInstanceUID
    GROUP BY i.collection_id
    ORDER BY slides DESC
""")

print(f"Collections with slide microscopy data: {len(sm_collections)}")
sm_collections.head(20)

### Select a slide from CPTAC-LUAD

We'll pick a slide from the `cptac_luad` (Clinical Proteomic Tumor Analysis Consortium - Lung Adenocarcinoma) collection. We'll look for a 20x slide with a relatively small file size to keep the download quick.

In [ ]:
candidates = idc_client.sql_query("""
    SELECT
        i.SeriesInstanceUID,
        i.StudyInstanceUID,
        i.PatientID,
        i.collection_id,
        ROUND(i.series_size_MB, 1) as size_mb,
        s.ObjectiveLensPower,
        s.max_TotalPixelMatrixColumns as width,
        s.max_TotalPixelMatrixRows as height,
        s.min_PixelSpacing_2sf as pixel_spacing_mm,
        i.license_short_name
    FROM sm_index s
    JOIN index i ON s.SeriesInstanceUID = i.SeriesInstanceUID
    WHERE i.collection_id = 'cptac_luad'
        AND s.ObjectiveLensPower = 20
    ORDER BY i.series_size_MB ASC
    LIMIT 10
""")

print("Smallest 20x slides from CPTAC-LUAD:")
candidates

In [ ]:
# Select the smallest slide
selected = candidates.iloc[0]
series_uid = selected['SeriesInstanceUID']
study_uid = selected['StudyInstanceUID']

print(f"Selected slide:")
print(f"  Patient: {selected['PatientID']}")
print(f"  Series UID: {series_uid}")
print(f"  Size: {selected['size_mb']} MB")
print(f"  Dimensions: {selected['width']} x {selected['height']} pixels")
print(f"  Objective Power: {selected['ObjectiveLensPower']}x")
print(f"  Pixel Spacing: {selected['pixel_spacing_mm']} mm")
print(f"  License: {selected['license_short_name']}")

## 2. Download the DICOM WSI from IDC

IDC stores pathology images as DICOM Slide Microscopy (SM) objects. Each slide is a DICOM Series containing multiple `.dcm` files (one per pyramid level/frame). We'll download into a folder named by the SeriesInstanceUID.

In [ ]:
download_dir = './slides'
os.makedirs(download_dir, exist_ok=True)

idc_client.download_from_selection(
    downloadDir=download_dir,
    seriesInstanceUID=[series_uid],
    dirTemplate='%SeriesInstanceUID'
)

slide_path = os.path.join(download_dir, series_uid)
dcm_files = [f for f in os.listdir(slide_path) if f.endswith('.dcm')]
print(f"Downloaded {len(dcm_files)} DICOM files to {slide_path}")

## 3. Open the Slide with TIAToolbox

TIAToolbox's `WSIReader.open()` automatically detects DICOM WSI directories and returns a `DICOMWSIReader`. This provides a unified API for reading slides regardless of the underlying format (DICOM, SVS, NDPI, etc.).

In [ ]:
reader = WSIReader.open(slide_path)

print(f"Reader type: {type(reader).__name__}")
print(f"\nSlide metadata:")

info = reader.info

# DICOMWSIReader may not populate objective_power or mpp from DICOM metadata.
# We can fill them in from the IDC sm_index query results.
if info.objective_power is None:
    info.objective_power = float(selected['ObjectiveLensPower'])
    print(f"  (objective_power set from IDC metadata)")

if info.mpp is None:
    # pixel_spacing_mm from sm_index is in mm; convert to microns
    pixel_spacing_um = float(selected['pixel_spacing_mm']) * 1000
    info.mpp = np.array([pixel_spacing_um, pixel_spacing_um])
    print(f"  (mpp set from IDC pixel spacing)")

print(f"  Slide dimensions (w x h): {info.slide_dimensions}")
print(f"  Objective power: {info.objective_power}")
print(f"  MPP (microns per pixel): {info.mpp}")
print(f"  Number of levels: {len(info.level_dimensions)}")
print(f"  Level dimensions: {info.level_dimensions}")
print(f"  Level downsamples: {info.level_downsamples}")
print(f"  Vendor: {info.vendor}")

## 4. Generate a Slide Thumbnail

A thumbnail gives us a quick overview of the entire slide. TIAToolbox can generate thumbnails at any target resolution.

In [ ]:
# Generate thumbnail at 1.25x magnification
thumbnail = reader.slide_thumbnail(resolution=1.25, units="power")

print(f"Thumbnail shape: {thumbnail.shape}")

fig, ax = plt.subplots(figsize=(10, 8))
ax.imshow(thumbnail)
ax.set_title(f"Slide Thumbnail ({selected['PatientID']}, 1.25x)", fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()

## 5. Read Regions at Different Resolutions

TIAToolbox provides two main methods for reading regions:

- **`read_bounds(bounds, resolution, units)`**: Reads a region defined by a bounding box in **baseline coordinates** (left, top, right, bottom) and returns it at the requested resolution. The output pixel dimensions change with resolution.
- **`read_rect(location, size, resolution, units)`**: Reads a fixed-size patch at the requested resolution. The location is in baseline coordinates by default.

Both support multiple resolution unit systems:
- **`units="power"`**: Magnification (e.g., 5x, 10x, 20x)
- **`units="mpp"`**: Microns per pixel (e.g., 0.5 mpp)
- **`units="level"`**: Pyramid level directly (0 = highest resolution)

**Note on DICOMWSIReader:** The current `DICOMWSIReader` (via `wsidicom`) has a coordinate handling issue when reading at resolutions that map to non-baseline pyramid levels — baseline coordinates are passed unscaled, causing out-of-bounds errors. A reliable workaround is to read at native resolution and resize, which we demonstrate below.

### Same region at different magnifications

In [ ]:
# Pick a location in the tissue area
# We'll find a tissue-rich region by looking at the thumbnail
thumb_gray = np.mean(thumbnail, axis=2)
tissue_mask = thumb_gray < 200  # Simple threshold to find tissue

# Find the center of mass of tissue
tissue_coords = np.argwhere(tissue_mask)
if len(tissue_coords) > 0:
    center_y, center_x = tissue_coords.mean(axis=0).astype(int)
else:
    center_y, center_x = thumbnail.shape[0] // 2, thumbnail.shape[1] // 2

# Scale from thumbnail coordinates to baseline (full resolution) coordinates
thumb_scale_x = info.slide_dimensions[0] / thumbnail.shape[1]
thumb_scale_y = info.slide_dimensions[1] / thumbnail.shape[0]
baseline_x = int(center_x * thumb_scale_x)
baseline_y = int(center_y * thumb_scale_y)

print(f"Tissue center at thumbnail coords: ({center_x}, {center_y})")
print(f"Tissue center at baseline coords: ({baseline_x}, {baseline_y})")

In [ ]:
# Define a region in baseline coordinates centered on the tissue
region_size = 2048  # baseline pixels
bounds = (
    max(0, baseline_x - region_size // 2),
    max(0, baseline_y - region_size // 2),
    min(info.slide_dimensions[0], baseline_x + region_size // 2),
    min(info.slide_dimensions[1], baseline_y + region_size // 2),
)

# Read the region at native resolution, then resize to show different magnifications
region_native = reader.read_bounds(
    bounds=bounds,
    resolution=info.objective_power,
    units="power",
)

powers = [5, 10, 20]
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, power in zip(axes, powers):
    scale = power / info.objective_power
    if scale < 1:
        new_w = int(region_native.shape[1] * scale)
        new_h = int(region_native.shape[0] * scale)
        region = np.array(Image.fromarray(region_native).resize((new_w, new_h), Image.LANCZOS))
    else:
        region = region_native

    ax.imshow(region)
    ax.set_title(f"{power}x magnification\n{region.shape[1]}x{region.shape[0]} pixels", fontsize=12)
    ax.axis('off')

plt.suptitle(f"Same tissue region ({region_size}x{region_size} baseline px) at different magnifications", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### Same region at different microns per pixel (mpp)

Microns per pixel is often the preferred unit in computational pathology because it's independent of the scanner's objective lens.

In [ ]:
# Same region at different microns-per-pixel resolutions
mpp_values = [0.5, 1.0, 2.0]
native_mpp = float(info.mpp[0])

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, mpp in zip(axes, mpp_values):
    scale = native_mpp / mpp
    if scale < 1:
        new_w = int(region_native.shape[1] * scale)
        new_h = int(region_native.shape[0] * scale)
        region = np.array(Image.fromarray(region_native).resize((new_w, new_h), Image.LANCZOS))
    else:
        region = region_native

    ax.imshow(region)
    ax.set_title(f"{mpp} mpp\n{region.shape[1]}x{region.shape[0]} pixels", fontsize=12)
    ax.axis('off')

plt.suptitle(f"Same tissue region at different microns-per-pixel resolutions", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### Reading a fixed-size patch with `read_rect()`

`read_rect()` reads a patch of a given pixel size at the requested resolution. The `location` is the top-left corner in baseline coordinates. This is useful when you need fixed-size patches (e.g., 256x256 or 512x512) for model input.

In [ ]:
# Read fixed-size patches at the native resolution using read_rect()
# Location is the top-left corner; coord_space="resolution" means coordinates
# are in the target resolution's pixel space (at native power, same as baseline)
native_power = int(info.objective_power)

# Ensure location is within bounds
loc_x = max(0, baseline_x - 512)
loc_y = max(0, baseline_y - 512)

patch_sizes = [256, 512, 1024]
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, size in zip(axes, patch_sizes):
    patch = reader.read_rect(
        location=(loc_x, loc_y),
        size=(size, size),
        resolution=native_power,
        units="power",
        coord_space="resolution",
    )

    ax.imshow(patch)
    ax.set_title(f"{size}x{size} patch at {native_power}x", fontsize=12)
    ax.axis('off')

plt.suptitle("Fixed-size patches from the same location using read_rect()", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 6. Visualize in the SLIM Viewer

IDC provides a browser-based slide viewer ([SLIM](https://github.com/ImagingDataCommons/slim)) that lets you explore the same slide interactively without any local software. This is useful for verifying what you see programmatically.

In [ ]:
viewer_url = idc_client.get_viewer_URL(
    seriesInstanceUID=series_uid
)

print(f"View this slide in SLIM viewer:")
print(viewer_url)

## 7. Explore Slide Metadata in Detail

Let's look at the full metadata dictionary available from TIAToolbox's reader.

In [ ]:
info_dict = reader.info.as_dict()
for key, value in info_dict.items():
    print(f"  {key}: {value}")

## Summary

In this notebook, we learned how to:

- Use `idc-index` with `sm_index` to discover and select slide microscopy data from IDC
- Download DICOM WSI files using `download_from_selection()` with `dirTemplate='%SeriesInstanceUID'`
- Open DICOM slides with `WSIReader.open()`, which automatically detects the format and returns a `DICOMWSIReader`
- Read metadata including dimensions, magnification, and microns-per-pixel
- Generate thumbnails and read regions using different resolution units (`power`, `mpp`)
- Use `read_rect()` for location+size based reading and `read_bounds()` for bounding-box based reading
- Get a SLIM viewer URL for interactive browser-based visualization

**Next:** [Notebook 02](02_tissue_masking_and_patch_extraction.ipynb) covers tissue detection and patch extraction from IDC slides.

## Acknowledgments

- **IDC:** Fedorov, A., et al. "National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence." *RadioGraphics* 43.12 (2023). https://doi.org/10.1148/rg.230180
- **TIAToolbox:** Pocock, J., et al. "TIAToolbox as an end-to-end library for advanced tissue image analytics." *Communications Medicine* 2, 120 (2022). https://doi.org/10.1038/s43856-022-00186-5